Ignis: Latent Dirichlet Allocation
============

In [1]:
import ignis

In [2]:
# Jupyter notebook setup
import ipywidgets as widgets
from IPython.core.display import display, HTML

# Custom styling:
# - Prevent vertical scrollbars in output subareas
# - Resize to fit pyLDAvis visualisations without causing other cells to overflow
style = """
<style>
   .jupyter-widgets-output-area .output_scroll {
        height: unset !important;
        border-radius: unset !important;
        -webkit-box-shadow: unset !important;
        box-shadow: unset !important;
    }
    .jupyter-widgets-output-area  {
        height: auto !important;
    }
</style>
<style>
    #notebook-container { width: 1370px !important; }
    div.output_area { width: unset !important; }
</style>
"""
display(HTML(style))

Model training (LDA)
----

Load from an `ignis.Corpus`, add the processed docs to an LDA model, and train it.

The random seed and parallelisation can both affect results, so setting the seed and number of workers is necessary for reproducibility.

In [3]:
corpus = ignis.load_corpus("bbc-tf-idf.corpus")

With the current public version of `pyLDAvis`, (2.1.2), preparing the visualisation data takes very long with recent versions of `pandas` (>0.23.4).  We have an option here to use an optimised version of the preparation function built into Ignis.

In [4]:
use_optimised = True

In [9]:
model_options = {
    "k": 5,
    "term_weighting": "idf",
    "until_max_ll": True,
    "verbose": True
}
vis_options = {"verbose": True, "use_optimised": use_optimised}
results = ignis.train_model(
    corpus,
    model_type="tp_lda",
    model_options=model_options,
    vis_type="pyldavis",
    vis_options=vis_options,
)

Training LDA model on 2119 documents:
{'term_weighting': 'idf', 'k': 5, 'seed': 11399, 'workers': 8, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': True, 'max_extra_iterations': 5000, 'verbose': True, 'tw': <TermWeight.IDF: 1>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:06<00:00, 146.60it/s, Log-likelihood=-32.87328]



Continuing to train until maximum log-likelihood.
(N.B.: This may not correlate with increased interpretability)



2000it [00:18, 107.50it/s, Log-likelihood=-32.66813]


Model training complete. (26.318s)
Preparing LDA visualisation... Done. (0.341s)


Graphical visualisation
--------

In [10]:
results.show_visualisation()

In [9]:
# results.export_visualisation("bbc_results")

## Saving results, using the automated labeller

In [11]:
results.save("bbc-full.aurum")
results = ignis.load_results("bbc-full.aurum")

In [12]:
results.init_labeller("tomotopy", verbose=True)

Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.


Exploring the top words and suggested labels for each topic
------

In [13]:
results.nb_show_topics(top_labels=15, top_words=15)

interactive(children=(IntSlider(value=1, description='topic_id', max=5, min=1), Output()), _dom_classes=('widg…

<function ignis.aurum.Aurum.nb_show_topics.<locals>.show_topic(topic_id=1)>

## Exploring documents that "belong" to a given topic

Because topics are distributions over words and documents are *distributions* over topics, documents don't belong to individual topics per se; every topic is represented in every document with some probability.

We therefore have to specify how many of the document's top `n` topics we want to check for the actual topic we're interested in.

This is especially significant for topic models that use a term weighting scheme, because all the common words (i.e., what we might consider stopwords) tend to get grouped into a single large topic; if we only consider each document's single most probable topic, we will unintentionally exclude documents which have this "stopwords" topic as their top topic.  (This is, however, not always a given; one would need to check the model's output and iterate from there)

In [16]:
# Check the topic that appears to be related to the entertainment industry
results.nb_show_topic_documents(topic_id=5, within_top_n=1)

interactive(children=(IntSlider(value=0, description='index', max=412), Output()), _dom_classes=('widget-inter…

<function ignis.aurum.Aurum.nb_show_topic_documents.<locals>.show_topic_doc(index=0)>

Slicing and iteration
--------
After seeing what the main topics might be, we can slice the initial corpus further and re-run topic modelling to get better resolution.

In [17]:
# Try zooming in on the entertainment topic
entertainment_slice = results.slice_by_topic(topic_id=5, within_top_n=1)
entertainment_model = results.retrain_model(corpus_slice=entertainment_slice)

Training LDA model on 413 documents:
{'term_weighting': 'idf', 'k': 5, 'seed': 11399, 'workers': 8, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': True, 'max_extra_iterations': 5000, 'verbose': True, 'tw': <TermWeight.IDF: 1>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:02<00:00, 457.00it/s, Log-likelihood=-28.52362]


Continuing to train until maximum log-likelihood.
(N.B.: This may not correlate with increased interpretability)




1800it [00:04, 375.45it/s, Log-likelihood=-28.41838]

Model training complete. (7.174s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.
Preparing LDA visualisation...

 Done. (0.316s)


In [18]:
entertainment_model.show_visualisation()

In [14]:
# entertainment_model.export_visualisation("bbc_results_2")

The position of each topic cluster on the graph is not intrinsically informative per se (being simply the result of some specified dimensionality-reducing technique), but if we want we can run the modelling algorithm with a different random seed and see if we get a more meaningful set of topics.

(We can also override any of the previously set options.)

In [19]:
model_options = {"seed": 7156}
entertainment_model_2 = entertainment_model.retrain_model(model_options=model_options)
entertainment_model_2.show_visualisation()

Training LDA model on 413 documents:
{'term_weighting': 'idf', 'k': 5, 'seed': 7156, 'workers': 8, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': True, 'max_extra_iterations': 5000, 'verbose': True, 'tw': <TermWeight.IDF: 1>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:02<00:00, 414.98it/s, Log-likelihood=-28.61604]


Continuing to train until maximum log-likelihood.
(N.B.: This may not correlate with increased interpretability)




3100it [00:07, 388.85it/s, Log-likelihood=-28.52951]

Model training complete. (10.572s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.
Preparing LDA visualisation...

 Done. (0.320s)


In [16]:
# entertainment_model_2.export_visualisation("bbc_results_2a")

Alternatively, we can slice the corpus by explicitly searching for documents that contain certain tokens, in case we want to be absolutely sure we got all the documents that mention certain words or phrases.

In [21]:
# Topics that seem to deal with films
film_topic = entertainment_model_2.slice_by_topic(topic_id=3, within_top_n=1)
film_topic_2 = entertainment_model_2.slice_by_topic(topic_id=5, within_top_n=1)

# Tokens that are related to films
by_token_slice = entertainment_model_2.slice_by_tokens(
    ["film", "films", "movie", "cinema"], include_root=True
)

concat_slice = film_topic.concat(film_topic_2, by_token_slice)

concat_model = entertainment_model_2.retrain_model(
    concat_slice, model_options=model_options
)
concat_model.show_visualisation()

Training LDA model on 325 documents:
{'term_weighting': 'idf', 'k': 5, 'seed': 7156, 'workers': 8, 'parallel_scheme': 'default', 'iterations': 2000, 'update_every': 100, 'until_max_ll': True, 'max_extra_iterations': 5000, 'verbose': True, 'tw': <TermWeight.IDF: 1>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 2000/2000 [00:04<00:00, 443.54it/s, Log-likelihood=-27.93133]


Continuing to train until maximum log-likelihood.
(N.B.: This may not correlate with increased interpretability)




300it [00:00, 414.36it/s, Log-likelihood=-27.93025]

Model training complete. (5.396s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.
Preparing LDA visualisation...

 Done. (0.313s)


In [18]:
# concat_model.export_visualisation("bbc_results_3")

In [24]:
# Topicrelated to film awards
concat_model.nb_show_topic_documents(topic_id=2, within_top_n=1)

interactive(children=(IntSlider(value=0, description='index', max=94), Output()), _dom_classes=('widget-intera…

<function ignis.aurum.Aurum.nb_show_topic_documents.<locals>.show_topic_doc(index=0)>